In [1]:
import string
import random
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from keybert import KeyBERT
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
from Levenshtein import distance

C:\Users\Saptarshi MT\anaconda3\envs\tf_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# First mention a text

In [2]:
text = 'Sun, star around which Earth and the other components of the solar system revolve. It is the dominant body of the system, constituting more than 99 percent of its entire mass. The Sun is the source of an enormous amount of energy, a portion of which provides Earth with the light and heat necessary to support life.The Sun is classified as a G2 V star, with G2 standing for the second hottest stars of the yellow G class—of surface temperature about 5,800 kelvins (K)—and the V representing a main sequence, or dwarf, star, the typical star for this temperature class. (G stars are so called because of the prominence of a band of atomic and molecular spectral lines that the German physicist Joseph von Fraunhofer designated G.) The Sun exists in the outer part of the Milky Way Galaxy and was formed from material that had been processed inside a supernova. The Sun is not, as is often said, a small star. Although it falls midway between the biggest and smallest stars of its type, there are so many dwarf stars that the Sun falls in the top 5 percent of stars in the neighbourhood that immediately surrounds it.'
text

'Sun, star around which Earth and the other components of the solar system revolve. It is the dominant body of the system, constituting more than 99 percent of its entire mass. The Sun is the source of an enormous amount of energy, a portion of which provides Earth with the light and heat necessary to support life.The Sun is classified as a G2 V star, with G2 standing for the second hottest stars of the yellow G class—of surface temperature about 5,800 kelvins (K)—and the V representing a main sequence, or dwarf, star, the typical star for this temperature class. (G stars are so called because of the prominence of a band of atomic and molecular spectral lines that the German physicist Joseph von Fraunhofer designated G.) The Sun exists in the outer part of the Milky Way Galaxy and was formed from material that had been processed inside a supernova. The Sun is not, as is often said, a small star. Although it falls midway between the biggest and smallest stars of its type, there are so m

# Second lemmatize all the words

In [3]:
lemmatizer = WordNetLemmatizer()

review = text
review = review.lower()
review = review.split()

review = [lemmatizer.lemmatize(word) for word in review]
review = ' '.join(review)

text = review
text

'sun, star around which earth and the other component of the solar system revolve. it is the dominant body of the system, constituting more than 99 percent of it entire mass. the sun is the source of an enormous amount of energy, a portion of which provides earth with the light and heat necessary to support life.the sun is classified a a g2 v star, with g2 standing for the second hottest star of the yellow g class—of surface temperature about 5,800 kelvin (k)—and the v representing a main sequence, or dwarf, star, the typical star for this temperature class. (g star are so called because of the prominence of a band of atomic and molecular spectral line that the german physicist joseph von fraunhofer designated g.) the sun exists in the outer part of the milky way galaxy and wa formed from material that had been processed inside a supernova. the sun is not, a is often said, a small star. although it fall midway between the biggest and smallest star of it type, there are so many dwarf st

# Now extract all the key-words from the text. 
# I used keyBERT which is a pretrained transformer architecture.

In [4]:
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(text,top_n=30)
print(keywords)

[('sun', 0.5244), ('solar', 0.5016), ('star', 0.446), ('supernova', 0.4075), ('milky', 0.4001), ('dwarf', 0.3457), ('galaxy', 0.3089), ('temperature', 0.2921), ('g2', 0.2612), ('kelvin', 0.2559), ('energy', 0.2164), ('heat', 0.2125), ('spectral', 0.2096), ('hottest', 0.2045), ('mass', 0.2022), ('formed', 0.186), ('earth', 0.1795), ('class', 0.1733), ('yellow', 0.1697), ('outer', 0.1669), ('light', 0.1653), ('molecular', 0.1631), ('atomic', 0.1594), ('main', 0.1558), ('prominence', 0.1536), ('smallest', 0.1408), ('material', 0.1393), ('classified', 0.133), ('biggest', 0.129), ('type', 0.1161)]


Extract the keys

In [5]:
filtered_keys = []
for i in range(len(keywords)):
    filtered_keys.append(keywords[i][0])
filtered_keys

['sun',
 'solar',
 'star',
 'supernova',
 'milky',
 'dwarf',
 'galaxy',
 'temperature',
 'g2',
 'kelvin',
 'energy',
 'heat',
 'spectral',
 'hottest',
 'mass',
 'formed',
 'earth',
 'class',
 'yellow',
 'outer',
 'light',
 'molecular',
 'atomic',
 'main',
 'prominence',
 'smallest',
 'material',
 'classified',
 'biggest',
 'type']

# Now match the keywords to the sentence they belong to

In [6]:
def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

sentences = tokenize_sentences(text)
keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)
        
print(keyword_sentence_mapping)

{'sun': ['the sun is the source of an enormous amount of energy, a portion of which provides earth with the light and heat necessary to support life.the sun is classified a a g2 v star, with g2 standing for the second hottest star of the yellow g class—of surface temperature about 5,800 kelvin (k)—and the v representing a main sequence, or dwarf, star, the typical star for this temperature class.', 'the sun is the source of an enormous amount of energy, a portion of which provides earth with the light and heat necessary to support life.the sun is classified a a g2 v star, with g2 standing for the second hottest star of the yellow g class—of surface temperature about 5,800 kelvin (k)—and the v representing a main sequence, or dwarf, star, the typical star for this temperature class.', '(g star are so called because of the prominence of a band of atomic and molecular spectral line that the german physicist joseph von fraunhofer designated g.) the sun exists in the outer part of the milky

In [7]:
keyword_sentence_mapping.keys()

dict_keys(['sun', 'solar', 'star', 'supernova', 'milky', 'dwarf', 'galaxy', 'temperature', 'g2', 'kelvin', 'energy', 'heat', 'spectral', 'hottest', 'mass', 'formed', 'earth', 'class', 'yellow', 'outer', 'light', 'molecular', 'atomic', 'main', 'prominence', 'smallest', 'material', 'classified', 'biggest', 'type'])

# Weed out similar type of words. I used Levenshtein distance

In [8]:
# Define the list of words
words =filtered_keys
# Define the threshold for similarity
threshold = 4

# Find pairs of words with Levenshtein distance below the threshold
similar_pairs = []
for i in range(len(words)):
    for j in range(i+1, len(words)):
        if distance(words[i], words[j]) <= threshold:
            similar_pairs.append((words[i], words[j]))

# Get the list of similar words
similar = list(set([word for pair in similar_pairs for word in pair]))
similar_words = [i for i in filtered_keys if i in similar]

# Print the similar words
print("Similar words in the list:", similar_words)

Similar words in the list: ['sun', 'solar', 'star', 'milky', 'dwarf', 'galaxy', 'g2', 'kelvin', 'energy', 'heat', 'hottest', 'mass', 'formed', 'earth', 'class', 'yellow', 'outer', 'light', 'main', 'biggest', 'type']


# Finally create distractors of the keywords. I used Wordnet. 
# Then generate MCQs.

In [9]:
for j in similar_words:
    
    for i in keyword_sentence_mapping[j]:
        
        target_word = j
        synsets = wn.synsets(target_word)

        # Get the hyponyms of the target word from each synset
        hyponyms = []
        for synset in synsets:
            hyponyms += synset.hyponyms()

        # Get the lemmas of the hyponyms
        lemmas = []
        for hyponym in hyponyms:
            lemmas += hyponym.lemmas()

        # Get the names of the lemmas and filter out the target word itself
        distractors = [lemma.name() for lemma in lemmas if lemma.name() != target_word]

        if len(distractors)>0:
            dist = distractors[:3]
            dist.append(j)
            random.shuffle(dist)
            print('Q. ',i.replace(j,'______'))
            print(' ')
            print('      you options are : ',dist)
            print('         ')
        

Q.  the ______ is the source of an enormous amount of energy, a portion of which provides earth with the light and heat necessary to support life.the ______ is classified a a g2 v star, with g2 standing for the second hottest star of the yellow g class—of surface temperature about 5,800 kelvin (k)—and the v representing a main sequence, or dwarf, star, the typical star for this temperature class.
 
      you options are :  ['sunburst', 'sun']
         
Q.  the ______ is the source of an enormous amount of energy, a portion of which provides earth with the light and heat necessary to support life.the ______ is classified a a g2 v star, with g2 standing for the second hottest star of the yellow g class—of surface temperature about 5,800 kelvin (k)—and the v representing a main sequence, or dwarf, star, the typical star for this temperature class.
 
      you options are :  ['sunburst', 'sun']
         
Q.  (g star are so called because of the prominence of a band of atomic and molecular 